### Guess the Car Model in an Image

The Image will be read in from a url.

In [2]:
import json
import os
import random
import numpy as np

import cv2 as cv
import keras.backend as K
import numpy as np
import scipy.io

from PIL import Image
import requests
from io import BytesIO

In [3]:
%run "/Users/evan@datainsights.de/resnets_utils"

In [4]:
%run "/Users/evan@datainsights.de/scale_layer"

In [5]:
%run "/Users/evan@datainsights.de/res_net"

### Import the Model and Load the Pre-Trained Weights
This could take a minute or two

In [7]:
img_width, img_height = 224, 224
model = load_model_weights()
model.load_weights('/dbfs/evan/weights.best.hdf5')

cars_meta = scipy.io.loadmat('/dbfs/evan/cars_meta')
class_names = cars_meta['class_names']  # shape=(1, 196)
class_names = np.transpose(class_names)

### Define the Guess Function
This reads in a url, and then puts the corresponding image through the neural network to make a prediction about the car

In [9]:
def guess(url):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))

    results = []
    img = cv.resize(np.float32(img), (img_width, img_height), cv.INTER_CUBIC)
    rgb_img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    rgb_img = np.expand_dims(rgb_img, 0)
    preds = model.predict(rgb_img)
    prob = np.max(preds)
    class_id = np.argmax(preds)
    text = ('Predict: {}, prob: {}'.format(class_names[class_id][0][0], prob))
    results.append({'label': class_names[class_id][0][0], 'prob': '{:.4}'.format(prob)})
    cv.imwrite('images/{}_out.png', img)

    return(results)

### Run the Guess Function with Multiple Input Images

In [11]:
url_list = ["http://cdn.shopify.com/s/files/1/1710/1473/products/LS0704-0915-3_1024x1024.jpg?v=1506931919",\
           "https://picolio.auto123.com/12photo/bmw/2012-bmw-m3_2.jpg",\
           "https://img2.carmax.com/img/vehicles/17975771/1.jpg",\
           "https://s3.ics-assets.co.uk/resized/media/img/car360_image_images/870-881337/P0oD2BMWRnAuC2kGQvYe.JPG",\
           "https://st.motortrend.com/uploads/sites/10/2015/09/2014-Aston-Martin-V8-Vantage-three-quarters-back-side-view-001.jpg"]
for url in url_list:
  print(guess(url))

In [12]:
K.clear_session()